# Normalizer Performance Analysis

This notebook contains an analysis of the normalizer performance on the CIViC, MOA, and Clinvar data

## Import relevant packages

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
# from civicpy import civic as civicpy
# import plotly.express as px
import ndjson
import re
from dotenv import load_dotenv
import os

## Dictionaries to map variants to categories and record category counts

In [2]:
civic_category_bins = {
    "Sequence Variants": "Sequence Variants",
    "Copy Number Variants": "Copy Number Variants"
}



moa_category_bins = {
    "Sequence Variants": "Sequence Variants",
    "Copy Number Variants": "Copy Number Variants",
    "Rearrangement Variants": "Rearrangement Variants",
    "Expression Variants": "Expression Variants",
    "Other Variants": "Other Variants"
}

# the values in this dictionary are lists of 4 integer values:
# [nomalized_count, unable_to_normalize_count, unsupported_count, total_count]
nomalized_count = 0
unable_to_normalize_count = 1
unsupported_count = 2
total_count = 3

category_counts = {
    "Copy Number Variants":[0,0,0,0],
    "Expression Variants":[0,0,0,0],
    "Other Variants": [0,0,0,0],
    "Rearrangement Variants":[0,0,0,0],
    "Sequence Variants":[0,0,0,0]
}

## CIViC



In order to score the normalizer's performance on the CIViC data, some cleaning is required.

First we need to read in the data that was ostensibly supported, get rid of varients with multiple type labels, and assign variant types to as  many of the entries as possible that have a "Not provided" value for civic_variant_types.

Read in .csv of normalized variants in CIVIC

In [3]:
civic_normalized_df = pd.read_csv("../civic/variation_analysis/able_to_normalize_queries.csv",sep = "\t")
print(civic_normalized_df.shape)
civic_normalized_df.head()
type(civic_normalized_df)

(1876, 7)


pandas.core.frame.DataFrame

Prune columns and add new column to flag as normalized.

In [4]:
pruned_civic_normalized_df = civic_normalized_df[["variant_id","query","query_type","civic_variant_types"]]
pruned_civic_normalized_df.insert(4,"normalization_status","normalized")
pruned_civic_normalized_df.head()

,variant_id,query,query_type,civic_variant_types,normalization_status
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,Stop Lost,normalized
1,1988,NC_000003.11:g.10191649A>T,genomic,Stop Lost,normalized
2,2488,3-10191647-T-G,genomic,Stop Lost,normalized
3,1986,NC_000003.11:g.10191648G>T,genomic,Stop Lost,normalized
4,1987,NC_000003.11:g.10191649A>G,genomic,Stop Lost,normalized


Repeat process with the variants that were unable to be normalized.

In [5]:
civic_not_normalized_df = pd.read_csv("../civic/variation_analysis/unable_to_normalize_queries.csv",sep = "\t")
print(civic_not_normalized_df.shape)
civic_not_normalized_df.head()
type(civic_not_normalized_df)

(80, 8)


pandas.core.frame.DataFrame

In [6]:
pruned_civic_not_normalized_df = civic_not_normalized_df[["variant_id","query","query_type","civic_variant_types"]]
pruned_civic_not_normalized_df.insert(4,"normalization_status","not_normalized")
pruned_civic_not_normalized_df.head()

,variant_id,query,query_type,civic_variant_types,normalization_status
0,748,MLH1 *757L,protein,Stop Lost,not_normalized
1,3718,AR A748V,protein,Not provided,not_normalized
2,3725,AR A765T,protein,Not provided,not_normalized
3,4485,ERBB2 A775_G776ins YVMA,protein,Not provided,not_normalized
4,248,TERT C228T,protein,Regulatory Region Variant,not_normalized


Merge these dfs

In [7]:
frames = [pruned_civic_normalized_df, pruned_civic_not_normalized_df]
civic_supported_df = pd.concat(frames)
civic_supported_df.shape

(1956, 5)

Making all queries in all caps to make it easier to account of untyped variants later on.

In [8]:
civic_supported_df["query"] = civic_supported_df["query"].apply(str.upper)

Checking variant types.  The single largest types is "Not provided".  
Most of these look like amino acid substitutions.
Defining a regex to detect these variants and assign "Missense Varaint" type to these variants.

In [9]:
civic_supported_df["civic_variant_types"].value_counts(dropna=False)

civic_variant_types
Not provided                                                 816
Missense Variant                                             815
Stop Gained                                                   60
Frameshift Truncation                                         36
Transcript Amplification                                      36
Inframe Deletion                                              35
Minus 1 Frameshift Variant;Frameshift Truncation              21
Inframe Insertion                                             17
Synonymous Variant                                            13
Splice Donor Variant                                          13
Splice Acceptor Variant                                        9
Missense Variant;Gain Of Function Variant                      9
Frameshift Variant                                             7
Stop Lost                                                      6
Conservative Inframe Deletion                                  6
Delin

If a variant does not have an assigned variant type in civic, it is a protein query, and the query matches a regex pattern associated with variant flagstitutions (such as "PTEN A126D"), then I am re-classifying them as a "Missense Variant" instead.

In [10]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("\S+\s+[A-Z]+\d+[A-Z|*]", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "protein") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Missense Variant", civic_supported_df["civic_variant_types"])

# civic_supported_df[0:20]
    # bool(re.match("\S+\s+[A-Z]+\d+[A-Z]","PTEN A126D"))

Doing so reduced the 816 untyped variants down to 70.
Checking the remaining weird variants.

In [19]:
untyped_variants = civic_supported_df[civic_supported_df["civic_variant_types"] == "Not provided"]
print(untyped_variants.head(20))

      variant_id                               query query_type  \
328         2514                     3-10183806-A-CC    genomic   
545         2290                      3-10188288-G-A    genomic   
1178        1882                      3-10188252-A-C    genomic   
1544        2461          NC_000003.11:G.10188323A>C    genomic   
1557        2504          NC_000003.11:G.10188196A>G    genomic   
1870         615  NC_000009.11:G.5070053_5070054INSG    genomic   
1871        3161                      3-10183878-G-A    genomic   
48          4542          DICER1 RNASE IIIB MUTATION    protein   

     civic_variant_types normalization_status  variant flag  
328         Not provided           normalized         False  
545         Not provided           normalized         False  
1178        Not provided           normalized         False  
1544        Not provided           normalized         False  
1557        Not provided           normalized         False  
1870        Not provided

Reassigning variants marked as {gene} Amplification as Transcript Amplification Variants

In [12]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("\S+\s+AMPLIFICATION", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "protein") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Transcript Amplification", civic_supported_df["civic_variant_types"])


Reassigning amino acid insertions, delins, and deletions as "Missense Variant", including a couple of variants that have a random space before or after the sequence operation like "INS"

In [13]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("\S+\s+[A-Z]+\d+_+[A-Z]+\d+INS+[A-Z]", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "protein") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Missense Variant", civic_supported_df["civic_variant_types"])


In [14]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("\S+\s+[A-Z]+\d+_+[A-Z]+\d+INS+\s+[A-Z]", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "protein") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Missense Variant", civic_supported_df["civic_variant_types"])

In [15]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("\S+\s+[A-Z]+\d+-+\d+\s+INS+[A-Z]", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "protein") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Missense Variant", civic_supported_df["civic_variant_types"])

In [16]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("\S+\s+[A-Z]+\d+_+[A-Z]+\d+DELINS+[A-Z]", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "protein") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Missense Variant", civic_supported_df["civic_variant_types"])


In [17]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("\S+\s+[A-Z]+\d+_+[A-Z]+\d+DEL", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "protein") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Missense Variant", civic_supported_df["civic_variant_types"])


And assigning missense types to a handful of remaining variants that are non-standard names for genomic and protein sequence variants

In [18]:
civic_supported_df["variant flag"] = civic_supported_df["query"].apply(lambda x: bool(re.match("\S+\s+P\.+[A-Z]+\d+[A-Z]", x)))

civic_supported_df["civic_variant_types"] = np.where((civic_supported_df["query_type"] == "protein") & (civic_supported_df["civic_variant_types"] == "Not provided") & (civic_supported_df["variant flag"]), "Missense Variant", civic_supported_df["civic_variant_types"])


Add category column to CIViC df.

Bin variants to categories.

For variants with multiple associated types:  If the 2+ types have a subset relationship (eg frameshift; frameshift truncation), reassign with the superset type (frameshift).  If the types are disjoint (eg: Trancript Variant; Loss of Function Variant), reassign with the type most closely  associated with the assayed data (Transcript Variant).


For variants without an associated type (approximately 1,632 entries) use regexes to assign types where possible.

Finally, assign variants to category bins based on type.

Split df by normalized/not_normalized/not_supported flags

For each df, Get CIViC Variant counts by category and add to counts dictionary

## MOA

Read MOA .csv file for Normalized variants

In [ ]:
moa_normalized_df = pd.read_csv("../moa/feature_analysis/able_to_normalize_queries.csv",sep = "\t")
print(moa_normalized_df.shape)
moa_normalized_df.head()
type(moa_normalized_df)

Get variant counts by category, update variant counts df 

In [ ]:
moa_normalized_category_counts = moa_normalized_df["category"].value_counts(dropna=False)
moa_normalized_category_counts.head()
indeces = moa_normalized_category_counts.index
for i in range(len(moa_normalized_category_counts)):
    variant = moa_normalized_category_counts.index[i]
    count = moa_normalized_category_counts[i]
    print(variant, count)
    target_category = moa_category_bins[variant]
    # print(target_category)
    category_counts[target_category][nomalized_count] += count
    category_counts[target_category][total_count] += count

for i in category_counts.items():
    print(i)


Repeat same process for variants that were supported but failed to normalize.

In [ ]:
moa_not_normalized_df = pd.read_csv("../moa/feature_analysis/unable_to_normalize_queries.csv",sep = "\t")
print(moa_not_normalized_df.shape)
moa_not_normalized_df.head()
type(moa_not_normalized_df)

In [ ]:
moa__not_normalized_category_counts = moa_not_normalized_df["category"].value_counts(dropna=False)
moa__not_normalized_category_counts.head()
indeces = moa__not_normalized_category_counts.index
for i in range(len(moa__not_normalized_category_counts)):
    variant = moa__not_normalized_category_counts.index[i]
    count = moa__not_normalized_category_counts[i]
    print(variant, count)
    target_category = moa_category_bins[variant]
    # print(target_category)
    category_counts[target_category][unable_to_normalize_count] += count
    category_counts[target_category][total_count] += count

for i in category_counts.items():
    print(i)


Repeat same process for variants that are unsupported.

In [ ]:
moa_not_supported_df = pd.read_csv("../moa/feature_analysis/not_supported_variants.csv",sep = "\t")
print(moa_not_supported_df.shape)
print(moa_not_supported_df.head())
type(moa_not_supported_df)
print(moa_not_supported_df["category"].value_counts(dropna=False))

In [ ]:
othervars = moa_not_supported_df[moa_not_supported_df["category"] == "Other Variants"]
# print(othervars.head())
type(othervars)
othervars.head

In [ ]:
moa__not_supported_category_counts = moa_not_supported_df["category"].value_counts(dropna=False)
moa__not_supported_category_counts.head()
indeces = moa__not_supported_category_counts.index
for i in range(len(moa__not_supported_category_counts)):
    variant = moa__not_supported_category_counts.index[i]
    count = moa__not_supported_category_counts[i]
    print(variant, count)
    target_category = moa_category_bins[variant]
    # print(target_category)
    category_counts[target_category][unsupported_count] += count
    category_counts[target_category][total_count] += count

for i in category_counts.items():
    print(i)


## ClinVar

Get df from ClinVar Analysis

Get Clinvar Variant counts by category, update variant counts df 

Output counts df

Generate figure(?)